In [140]:
import numpy as np 
import pandas as pd 
from os import listdir
from shutil import copy2 
import regex as re 

# Pandas setup
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.max_colwidth', -1) 

In [2]:
path = "./Normalized/"
matches = listdir(path) 

matches

['surfactantes.csv',
 'pH.csv',
 'fosforo.csv',
 'toda_consema.csv',
 'alcalinidade.csv',
 'compostagem.csv',
 'ptel.csv',
 'Cor_verdadeira.csv',
 'DQO.csv',
 'DBO.csv',
 'coliformes.csv',
 'nitrogênio.csv',
 'leito_de_secagem.csv',
 'sólidos.csv',
 'óleos.csv',
 'acidez.csv']

In [126]:
df = pd.DataFrame()
for i in range(0, len(matches) - 1):
    df_1 = pd.read_csv(path + matches[i], encoding='utf8')
    df_2 = pd.read_csv(path + matches[i + 1], encoding='utf8')
    partial = df_1.append(df_2)
    df = df.append(partial)

df["CIANETO"] = [x[0] if str(x[0]) != "nan" else x[1] for x in zip(df["CIANETO"], df["CIANETO.1"])]
df["MERCÚRIO"] = [x[0] if str(x[0]) != "nan" else x[1] for x in zip(df["MERCÚRIO"], df["MERCÚRIO.1"])]
df["FENOL"] = [x[0] if str(x[0]) != "nan" else x[1] for x in zip(df["FENOL"], df["FENÓIS"])]
df["SÓLIDOS"] = [x[0] if str(x[0]) != "nan" else x[1] for x in zip(df["SÓLIDOS"], df["SÓLIDOS_VOLÁTEIS"])]

df.drop(columns=["CIANETO.1", "MERCÚRIO.1", "FENÓIS", "SÓLIDOS_VOLÁTEIS"], inplace=True)
df.drop([])
df.head()

,ENTIDADE,DATA,SURFACTANTES,PH,FÓSFORO,ALUMÍNIO,ARSÊNIO,BÁRIO,BORO,CÁDMIO,CHUMBO,CIANETO,COBALTO,COBRE,CROMO,CROMO_HEXAVALENTE,DBO,DQO,ESTANHO,FERRO,FLUORETO,LÍTIO,MANGANÊS,MERCÚRIO,MOLIBDÊNIO,NÍQUEL,NITROGÊNIO_AMONIACAL,NITROGÊNIO_KJELDAHL,COLIFORMES_TERMOTOLERANTES,PRATA,ÍNDICE_DE_FENÓIS,NITROGÊNIO,FENOL,ALCALINIDADE,UMIDADE,PORCENTAGEM_SÓLIDOS,COMPOSTOS,COR_VERDADEIRA,SÓLIDOS,ÓLEOS,ACIDEZ
0,P1,2013-01-22,1.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P2,2013-01-22,0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P7,2013-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P1,2013-02-05,0.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P2,2013-02-05,0.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
tidy = df.drop_duplicates()
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace(".1", ""))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace(".1", ""))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace("ANAMB", "ANAMBI"))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace("ANAMBII", "ANAMBI"))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace("SANEAMB", "ANAMBI"))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace("itt Fossil", "ITT FOSSIL"))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace("itt Fossil / NUCMAT*", "ITT FOSSIL"))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace("ITT FOSSIL / NUCMAT*", "ITT FOSSIL"))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: x.replace("Lab. Quimica e Farmácia", "LAB. QUIMICA"))
tidy.ENTIDADE = tidy.ENTIDADE.map(lambda x: re.sub(r"\s+", " ", x))

entities = tidy.ENTIDADE.unique().tolist()
tables = [tidy[tidy["ENTIDADE"] == entity] for entity in entities]

for table in tables:
    name = table.ENTIDADE.unique()[0].replace(" ", "_").lower()
    name = name.replace("/", "")
    name = name.replace(".", "")
    path = "./Entities/"
    table.to_csv(f"{path}{name}", encoding="utf8", header=True, index=False)

tidy.to_csv(f"{path}entire_dataset", encoding="utf8", index=False)